## Measures/Tests of Distribution
- central limit theorem
- t-test
    - one sample
        - one tailed: measuring directionality (divide p in half)
            >>
        - two tailed: measuring difference (p is the correct value)
            >>
    - two sample

## Measures of Correlation
- Pearson r-test
    - measures correlation between two variables, in units of "r". But the measure is unitless, inherently
    - only tests Linear relationships
        >> **r, p = stats.pearson(x,y)**
            - where r is correlation, p is probability that relationship is due to chance
            - conceptually: the confidence interval above and below the line of best fit. +/- implies relationship
    